In [1]:
import time
import os
import json
import random
import numpy as np
import librosa
import hashlib
import sounddevice as sd
import pickle as pkl
import dill
import datetime
import matplotlib.pyplot as plt
from torchsummary import summary

import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import DataLoader

from src.resnet import ResNet
from src.dataset import GoogleSpeechCommandsDataset
from src.train import test

/usr/lib/python3/dist-packages/paramiko/transport.py:220: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Parameters
samples = 20
halving = False                     # halving = True iteratively prune | halving = True: equal pruning interval
halving_amount = 0.25
compare_with_random = True

use_snn = False
use_cpu = False
path = 'models/'
model_name = 'neglect_resnet_ep59_acc93_sprs0.pt' if use_snn else 'neglect_resnet_ep59_acc93_sprs0.pt'
base_model_name = 'resnnet_ep47_acc72_sprs68.pt' if use_snn else 'resnet_ep55_acc99_sprs0.pt'
data_test_dir = '/home/marcel/Source/Python/DLOptimization/data/speech_commands_test_set_v0.02'
data_cache_dir = '/home/marcel/Source/Python/DLOptimization/cache/data/'
load_data_from_dir = None

In [3]:
# Load model
torch.set_flush_denormal(True)
device = torch.device("cuda" if not use_cpu and torch.cuda.is_available() else "cpu")
model_path = str(os.path.join(path, model_name))
model = ResNet.from_state_dict(torch.load(model_path), neglect=0.5)
model.neglect = True
base_model_path = str(os.path.join(path, base_model_name))
base_model = ResNet.from_state_dict(torch.load(base_model_path), spiking=True)
model.to(device)
base_model.to(device)
criterion = nn.CrossEntropyLoss()

/home/marcel/Source/Python/EfficientNN/src/resnet.py:517: UserWarning: Model configuration was empty! Using default configuration
  warnings.warn('Model configuration was empty! Using default configuration')
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


In [4]:
summary(base_model, input_size=(1, 128, 111))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1          [-1, 1, 128, 111]               0
            Conv2d-2        [-1, 128, 128, 111]           1,152
       BatchNorm2d-3        [-1, 128, 128, 111]             256
             Leaky-4        [-1, 128, 128, 111]               0
            Conv2d-5        [-1, 128, 128, 111]         147,456
       BatchNorm2d-6        [-1, 128, 128, 111]             256
       DeQuantStub-7        [-1, 128, 128, 111]               0
             Leaky-8        [-1, 128, 128, 111]               0
         QuantStub-9        [-1, 128, 128, 111]               0
           Conv2d-10        [-1, 128, 128, 111]         147,456
      BatchNorm2d-11        [-1, 128, 128, 111]             256
      DeQuantStub-12        [-1, 128, 128, 111]               0
      DeQuantStub-13        [-1, 128, 128, 111]               0
            Leaky-14        [-1, 128, 1

In [5]:
# Load data
test_data = GoogleSpeechCommandsDataset(data_test_dir, data_cache_dir, encoder='mel', augment=0, train=False)
test_data.precache()
test_loader = DataLoader(test_data, batch_size=8, shuffle=True)

/home/marcel/Source/Python/Sketchbook/venv/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


--- Test Dataset ---
Path:	/home/marcel/Source/Python/DLOptimization/data/speech_commands_test_set_v0.02
Cache:	/home/marcel/Source/Python/DLOptimization/cache/data/speech_commands_test_set_v0.02
Labels:	('up', 'down', 'left', 'right', 'go', 'stop', 'yes', 'no', 'on', 'off')
Length:	4074


In [6]:
(base_accuracy, base_sparsity, base_significance, 
 base_gpu_time, base_cpu_time_mean, base_cpu_time_std, base_num_params) = test(base_model, test_loader, criterion, 
                                                                               cpu_tests=12, return_time=True, log_interval=512,
                                                                               neglect_threshold=1, num_steps=5, device=device)


--- Test Results ---
-> Accuracy Validation on CUDA
Test [0/4074 (0%)]	Correct: 2/4074	ACC: 0.05%	Epoch: None
-> Performance Testing on CPU
	-> Model caching...
	-> CPU time testing...
		1. run (4.6867s)
		2. run (3.4374s)
		3. run (3.1906s)
		4. run (3.4436s)
		5. run (3.2285s)
		6. run (3.3021s)
		7. run (3.5367s)
		8. run (3.1262s)
		9. run (3.3959s)
		10. run (3.353s)
		11. run (3.3715s)
		12. run (7.1858s)


/home/marcel/Source/Python/EfficientNN/src/train.py:185: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(data, dtype=torch.float32)



	Avg. Loss:		2.303444
	Accuracy:		10.43 % (425/4074)
	Sparsity:		0.00 %
	Avg. Weight:	0.01
	Num. Params:	58766410
	Model Size:		47.12 Mb 
	Device Time:	0.035008 s
	CPU Time:		3.7715±1.15 s


In [ ]:
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)
values = {}
for th in [0., 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.]:
    accuracy, sparsity, significance, gpu_time, cpu_time_mean, cpu_time_std, num_params = test(model, test_loader, criterion, 
                                                                                               cpu_tests=12, return_time=True, log_interval=512,
                                                                                               neglect_threshold=th, num_steps=5, device=device)
    values['accuracy'] = accuracy
    values['sparsity'] = sparsity
    values['significance'] = significance
    values['gpu_time'] = gpu_time
    values['cpu_time_mean'] = cpu_time_mean
    values['cpu_time_std'] = cpu_time_std
    values['num_params'] = num_params
    values['stats'] = model.classifier_stats()


--- Test Results ---
-> Accuracy Validation on CUDA
Test [0/4074 (0%)]	Correct: 0/4074	ACC: 0.00%	Epoch: None
Test [512/4074 (13%)]	Correct: 87/4074	ACC: 2.14%	Epoch: None
Test [1024/4074 (25%)]	Correct: 179/4074	ACC: 4.39%	Epoch: None
Test [1536/4074 (38%)]	Correct: 275/4074	ACC: 6.75%	Epoch: None
Test [2048/4074 (50%)]	Correct: 352/4074	ACC: 8.64%	Epoch: None
Test [2560/4074 (63%)]	Correct: 449/4074	ACC: 11.02%	Epoch: None
Test [3072/4074 (75%)]	Correct: 559/4074	ACC: 13.72%	Epoch: None
Test [3584/4074 (88%)]	Correct: 649/4074	ACC: 15.93%	Epoch: None
-> Performance Testing on CPU
	-> Model caching...
	-> CPU time testing...
		1. run (0.15359s)
		2. run (0.14312s)
		3. run (0.15323s)
		4. run (0.19882s)
		5. run (0.15766s)
		6. run (0.19364s)
		7. run (0.14524s)
		8. run (0.19759s)
		9. run (0.19256s)
		10. run (0.21997s)
		11. run (0.23801s)
		12. run (0.21026s)

	Avg. Loss:		2.383104
	Accuracy:		18.04 % (735/4074)
	Sparsity:		0.00 %
	Avg. Weight:	0.02
	Num. Params:	56931739
	Model 

### BASELINE
Avg. Loss:		0.029475
Accuracy:		99.21 % (4042/4074)
Sparsity:		0.00 %
Avg. Weight:	0.01
Num. Params:	58766410
Model Size:		47.10 Mb 
Device Time:	0.020584 s
CPU Time:		0.6140±0.05 s



### TH = 0.25
Avg. Loss:		2.365174
Accuracy:		18.73 % (763/4074)
Sparsity:		0.00 %
Avg. Weight:	0.02
Num. Params:	56931739
Model Size:		45.98 Mb 
Device Time:	0.006546 s
CPU Time:		0.2371±0.00 s

'classifier1': 0, 'classifier2': 3997, 'classifier3': 72, 'classifier4': 5

### TH = 0.5
Avg. Loss:		0.993332
Accuracy:		68.78 % (2802/4074)
Sparsity:		0.00 %
Avg. Weight:	0.02
Num. Params:	56931739
Model Size:		45.98 Mb 
Device Time:	0.005911 s
CPU Time:		0.2288±0.00 s

'classifier1': 0, 'classifier2': 5295, 'classifier3': 847, 'classifier4': 5989

### TH = 0.75
Avg. Loss:		0.358834
Accuracy:		88.59 % (3609/4074)
Sparsity:		0.00 %
Avg. Weight:	0.02
Num. Params:	56931739
Model Size:		45.98 Mb 
Device Time:	0.009075 s
CPU Time:		0.3754±0.07 s

'classifier1': 0, 'classifier2': 97, 'classifier3': 214, 'classifier4': 3679


### TH = 1.0
Avg. Loss:		0.477591
Accuracy:		93.03 % (3790/4074)
Sparsity:		0.00 %
Avg. Weight:	0.02
Num. Params:	56931739
Model Size:		45.98 Mb 
Device Time:	0.006547 s
CPU Time:		0.4107±0.00 s

'classifier1': 0, 'classifier2': 5295, 'classifier3': 847, 'classifier4': 9094

